# 03 · Clustering des pays participants

Notebook orienté segmentation des pays pour identifier des profils homogènes à partir des données préparées.

## Objectifs
- Charger les agrégations pays/édition produites lors du préprocessing.
- Sélectionner et normaliser les variables pertinentes pour le clustering.
- Déterminer le nombre de clusters via la méthode du coude et la silhouette.
- Entraîner un modèle KMeans (baseline) et visualiser les groupes (PCA 2D).
- Exporter les clusters pour réutilisation dans la webapp et les rapports.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

sns.set_theme(style='whitegrid', context='talk')
np.random.seed(42)

In [ ]:
BASE_DIR = Path('..').resolve()
DATA_DIR = BASE_DIR / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
REPORTS_FIG_DIR = BASE_DIR / 'reports' / 'figures'
REPORTS_FIG_DIR.mkdir(parents=True, exist_ok=True)

processed_path = PROCESSED_DIR / 'country_year_summary.csv'
processed_path

In [ ]:
if processed_path.exists():
    country_year_df = pd.read_csv(processed_path)
else:
    raise FileNotFoundError('country_year_summary.csv introuvable. Exécuter 02_preprocessing.ipynb en premier.')

country_year_df.head(3)

## Préparation des features
On sélectionne les colonnes quantitatives utiles, on gère les valeurs manquantes et on standardise.

In [ ]:
feature_cols = ['medals_total', 'athletes_unique', 'avg_rank']

feature_df = country_year_df.dropna(subset=feature_cols).copy()
feature_df['avg_rank'] = feature_df['avg_rank'].fillna(feature_df['avg_rank'].median())

scaler = StandardScaler()
scaled_features = scaler.fit_transform(feature_df[feature_cols])
scaled_features[:3]

## Méthode du coude
On recherche une valeur de K pertinente en observant l'inertie.

In [ ]:
k_range = range(2, 11)
inertias = []
silhouette_scores = []

for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init='auto')
    labels = km.fit_predict(scaled_features)
    inertias.append(km.inertia_)
    silhouette_scores.append(silhouette_score(scaled_features, labels))

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
ax[0].plot(list(k_range), inertias, marker='o')
ax[0].set_title('Méthode du coude')
ax[0].set_xlabel('k')
ax[0].set_ylabel('Inertie')

ax[1].plot(list(k_range), silhouette_scores, marker='o', color='orange')
ax[1].set_title('Score de silhouette')
ax[1].set_xlabel('k')
ax[1].set_ylabel('Silhouette moyenne')
plt.tight_layout()
fig.savefig(REPORTS_FIG_DIR / 'clustering_elbow_silhouette.png', dpi=120)
fig

## Clustering final
Sélectionner une valeur de K (ici 4 par défaut, ajustable selon les courbes).

In [ ]:
BEST_K = 4  # à ajuster après lecture des graphiques
kmeans = KMeans(n_clusters=BEST_K, random_state=42, n_init='auto')
cluster_labels = kmeans.fit_predict(scaled_features)

feature_df['cluster'] = cluster_labels
feature_df.head(3)

## Visualisation PCA
Projection 2D pour représenter les clusters.

In [ ]:
pca = PCA(n_components=2, random_state=42)
pca_coords = pca.fit_transform(scaled_features)
feature_df['pca_1'] = pca_coords[:, 0]
feature_df['pca_2'] = pca_coords[:, 1]

fig, ax = plt.subplots(figsize=(10, 7))
sns.scatterplot(data=feature_df, x='pca_1', y='pca_2', hue='cluster', palette='tab10', ax=ax, s=80)
ax.set_title('Clusters de pays (projection PCA)')
plt.tight_layout()
fig.savefig(REPORTS_FIG_DIR / 'clustering_pca.png', dpi=120)
fig

## Profil des clusters
Statistiques descriptives pour interpréter chaque groupe.

In [ ]:
cluster_summary = (feature_df.groupby('cluster')[feature_cols]
                             .round(2))
cluster_summary

In [ ]:
top_countries_by_cluster = feature_df.sort_values('medals_total', ascending=False).groupby('cluster').head(5)
top_countries_by_cluster[['country_name', 'slug_game', 'medals_total', 'athletes_unique', 'avg_rank', 'cluster']]

## Export des résultats
Sauvegarde du dataset enrichi pour les notebooks suivants ou la webapp.

In [ ]:
clusters_output = PROCESSED_DIR / 'country_year_clusters.csv'
feature_df.to_csv(clusters_output, index=False)
clusters_output

## Pistes d'amélioration
- Tester d'autres variables (PIB, population, nombre de disciplines représentées).
- Comparer avec d'autres algorithmes (AgglomerativeClustering, DBSCAN) pour vérifier la robustesse.
- Valider la stabilité des clusters dans le temps (par période ou saison).
- Intégrer une évaluation plus fine (silhouette par cluster, Davies-Bouldin).